In [1]:
import os
from PIL import Image

In [10]:
src = '/home/sid/code/calibration_code/calibration'
subdirs = [x[0] for x in os.walk(src)]
rgb_dirs = [x for x in subdirs if x.endswith('rgb')]
omni_dirs = [x for x in subdirs if x.endswith('omni')]

79


In [ ]:
def crop_image(img, actual_width=512, actual_height=424):
    # Crop image to remove black borders - Function for omnidirectional images
    left_width = (img.width - actual_width) // 2
    right_width = top_x + actual_width

    top_height = (img.height - actual_height) // 2
    bottom_height = top_y + actual_height

    img = img.crop((left_width, top_height, right_width, bottom_height))
    return img


def resize_image(img, type='rgb'):
    # Resize RGB images to 1280x720
    # Resize Omni images to 1900x something
    if type == 'rgb':
        if img.size != (1280, 720):
            img = img.thumbnail((1280, 720),Image.ANTIALIAS) # Apparently preserves aspect ratio (need to test)
            # img = img.resize((1280, 720), Image.ANTIALIAS)

    elif type == 'omni':
        img = crop_image(img, actual_width=1200, actual_height=900)
    
    elif type == 'depth':
        if img.size != (512, 424):
            img = img.thumbnail((512, 424), Image.ANTIALIAS)

    elif type == 'ir':
        img = crop_image(img, actual_width=1200, actual_height=900)
        # img = img.resize((512, 424), Image.ANTIALIAS)

    else:
        raise ValueError('Invalid image type')

    return img

In [11]:
for dir in rgb_dirs:    
    dst = os.path.join(dir, 'resized')
    if not os.path.isdir(dst):
        os.mkdir(dst)
    files = [x for x in os.listdir(dir) if x.endswith('.jpg')]
    for file in files:
        img = Image.open(os.path.join(dir, file))
        img = resize_image(img)

['/home/sid/code/calibration_code/calibration',
 '/home/sid/code/calibration_code/calibration/calibration',
 '/home/sid/code/calibration_code/calibration/sync',
 '/home/sid/code/calibration_code/calibration/docs',
 '/home/sid/code/calibration_code/calibration/.git',
 '/home/sid/code/calibration_code/calibration/.git/info',
 '/home/sid/code/calibration_code/calibration/.git/logs',
 '/home/sid/code/calibration_code/calibration/.git/logs/refs',
 '/home/sid/code/calibration_code/calibration/.git/logs/refs/remotes',
 '/home/sid/code/calibration_code/calibration/.git/logs/refs/remotes/origin',
 '/home/sid/code/calibration_code/calibration/.git/logs/refs/heads',
 '/home/sid/code/calibration_code/calibration/.git/branches',
 '/home/sid/code/calibration_code/calibration/.git/objects',
 '/home/sid/code/calibration_code/calibration/.git/objects/56',
 '/home/sid/code/calibration_code/calibration/.git/objects/a3',
 '/home/sid/code/calibration_code/calibration/.git/objects/b1',
 '/home/sid/code/cali